In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd '/content/drive/MyDrive/student_resource 3/' #RECOMMENDED TO MOUNT YOUR GOOGLE DRIVE AND SAVE THE OUTPUT FILE INSIDE THE DRIVE.

Mounted at /content/drive


In [10]:
TEST_FILE_LOCATION = 'test_pt1.csv' # name/location of the test.csv file
OUTPUT_FILE_NAME = 'output_pt1_1.csv' # name/location of the output.csv file

In [3]:
!pip install paddlepaddle-gpu paddleocr #ocr model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=b5b14869b87d76844bc3e65724981f93166ec6d8f5f6f128f5470b3d1a0d5270
  Stored in di

In [4]:
from PIL import Image
import requests
from io import BytesIO
import pandas as pd
from paddleocr import PaddleOCR
import numpy as np
import matplotlib.pyplot as plt
import random
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import paddle

ocr = PaddleOCR(use_angle_cls=True, use_gpu=True, lang='en')

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:00<00:00, 4.65MiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:00<00:00, 10.5MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:00<00:00, 3.08MiB/s]

[2024/09/15 06:23:31] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

In [5]:
print(paddle.is_compiled_with_cuda()) #Should be True for ocr to work with gpu

True


#Algorithm

In [6]:
entity_unit_map = {
  "width": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "depth": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "height": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "item_weight": {
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  },
  "maximum_weight_recommendation": {
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  },
  "voltage": {
    "millivolt",
    "kilovolt",
    "volt"
  },
  "wattage": {
    "kilowatt",
    "watt"
  },
  "item_volume": {
    "cubic foot",
    "microlitre",
    "cup",
    "fluid ounce",
    "centilitre",
    "imperial gallon",
    "pint",
    "decilitre",
    "litre",
    "millilitre",
    "quart",
    "cubic inch",
    "gallon"
  }
}


map_queries = {
      'width' : 0,
      'depth' : 0,
      'height' : 0,
      'item_weight' : 1,
      'maximum_weight_recommendation' : 1,
      'voltage' : 2,
      'wattage' : 3,
      'item_volume' : 4
  }
unit_rank_maps = [
      {
      'millimetre': 1,
      'centimetre': 2,
      'inch': 3,
      'foot': 4,
      'yard': 5,
      'metre': 6
  },
      {
          'microgram': 1,
          'milligram': 2,
          'gram': 3,
          'ounce': 4,
          'pound': 5,
          'kilogram': 6,
          'ton': 7
      },
      {
          'millivolt': 1,
          'volt': 2,
          'kilovolt': 3
      },
      {
          'watt': 1,
          'kilowatt': 2
      },
      {
          'microlitre': 1,
          'millilitre': 2,
          'centilitre': 3,
          'decilitre': 4,
          'fluid ounce': 5,
          'cup': 6,
          'pint': 7,
          'quart': 8,
          'litre': 9,
          'gallon': 10,
          'imperial gallon': 11,
          'cubic inch': 12,
          'cubic foot': 13
      }
  ]


unit_abbreviation_maps = [
      {
          'centimetre': ['cm', 'cms', 'centimetres'],
          'foot': ['ft', 'feet', 'foot'],
          'millimetre': ['mm', 'millimetres'],
          'metre': ['m', 'metres'],
          'inch': ['in', 'inches', 'inch'],
          'yard': ['yd', 'yards', 'yard']
      },

        {
          'milligram': ['mg', 'milligrams'],
          'kilogram': ['kg', 'kgs', 'kilograms'],
          'microgram': ['µg', 'micrograms'],
          'gram': ['g', 'grams', 'gms'],
          'ounce': ['oz', 'ounces', 'ozs'],
          'ton': ['t', 'tons', 'tonnes'],
          'pound': ['lb', 'pounds', 'lbs'],
        },

      {'millivolt': ['mv', 'millivolts'],
      'kilovolt': ['kv', 'kilovolts'],
      'volt': ['v', 'volts']},

      {'kilowatt': ['kw', 'kilowatts'],
      'watt': ['w', 'watts']},

      {'cubic foot': ['cu ft', 'cubic feet', 'cu feet', 'cu foot'],
      'microlitre': ['µl', 'microlitres'],
      'cup': ['cups'],
      'fluid ounce': ['fl oz', 'fluid ounces', 'fl ounces', 'fl', 'fl ounce'],
      'centilitre': ['cl', 'centilitres', 'cls'],
      'imperial gallon': ['imp gal', 'imperial gallons'],
      'pint': ['pt', 'pints'],
      'decilitre': ['dl', 'decilitres'],
      'litre': ['l', 'litres'],
      'millilitre': ['ml', 'millilitres'],
      'quart': ['qt', 'quarts', 'q'],
      'cubic inch': ['cu in', 'cubic inches', 'cu inches', 'cu'],
      'gallon': ['gal', 'gallons']}
  ]
def predict(image_link, query):
  response = requests.get(image_link)
  img = Image.open(BytesIO(response.content))
  img = np.array(img)
  resa = ocr.ocr(img)

  sentence = ''
  if len(resa) == 0:
    return ""
  if resa[0] is None:
    return ""

  for line in resa[0]:
      sentence += line[1][0] + " "


  def generate_unit_pattern(entity_unit_map, unit_abbreviation_map):
      pattern_parts = []
      for key in unit_abbreviation_map.keys():
          unit_variants = [f'{key}']
          abbrev_units = [f'{x}' for x in unit_abbreviation_map[key]]
          all_units = unit_variants + abbrev_units
          pattern_parts.append(f"({'|'.join(all_units)})")
      return '|'.join(pattern_parts)

  def parse_measurements(text, value_unit_pattern, unit_abbreviation_map, unit_rank_map):
      flag = False
      text = text.lower()
      matches = value_unit_pattern.findall(text)
      result = []
      parsed_measurements = []

      for match in matches:
          start_value = float(match[0])
          end_value = float(match[3]) if match[3] else None
          unit = match[5]

          d = unit_abbreviation_map
          for key in d.keys():
            if unit in d[key]:
              unit = key

          if end_value:
              if end_value > start_value:
                  start_value = end_value
          result.append(f'{float(start_value)} {unit}')
          parsed_measurements.append((float(start_value), unit_rank_map[unit]))

      if len(result) == 0:
        flag = True
        number_matches = re.findall(r'\d+', text)
        print('Numbers found:', ', '.join(number_matches))
        result = number_matches

      return result, flag, parsed_measurements



  mapped_query = map_queries[query]
  unit_abbreviation_map = unit_abbreviation_maps[mapped_query]
  unit_rank_map = unit_rank_maps[mapped_query]

  unit_pattern = generate_unit_pattern(entity_unit_map, unit_abbreviation_map)
  value_unit_pattern = re.compile(r'(\d+(\.\d+)?)(-(\d+(\.\d+)?))?\s*(' + unit_pattern + r')', re.IGNORECASE)


  parsed_output, flag, parsed_measurements = parse_measurements(sentence, value_unit_pattern, unit_abbreviation_map, unit_rank_map)
  print(parsed_output)




  if len(parsed_output)==0:
    return ""
  else:
    if flag:
      return random.choice(parsed_output) + ' ' + random.choice(list(unit_abbreviation_maps[mapped_query].keys()))
    else:
      if len(parsed_output) == 1:
        return parsed_output[0]
      else:
        zipped_lists = list(zip(parsed_measurements, parsed_output))
        sorted_zipped_lists = sorted(zipped_lists, key=lambda x: (x[0][1], -x[0][0]))
        parsed_measurements, result = zip(*sorted_zipped_lists)
        result = list(result)
        if query == 'width':
          if len(result) > 1:
            return result[1]
          else:
            return result[-1]
        elif query == 'depth':
          if len(result) > 2:
            return result[2]
          else:
            return result[-1]
        elif query == 'height':
          return result[0]
        elif query == 'item_weight':
          return result[0]
        elif query == 'maximum_weight_recommendation':
          return result[0]
        elif query == 'voltage':
          return result[0]
        elif query == 'wattage':
          return result[0]
        elif query == 'item_volume':
          return result[0]

###Prediction

In [16]:
# code to restart the predictions from the last index in case runtime disconnects/fails

def get_last_index(output_file):
    try:
        df = pd.read_csv(output_file)
        last_index = df['index'].iloc[-1]
        return last_index + 1
    except (FileNotFoundError, IndexError) as E:
        print(E)
        return -1


df = pd.read_csv(OUTPUT_FILE_NAME)
print(df.tail(1)) #CHECK LAST INDEX


X_test = pd.read_csv(TEST_FILE_LOCATION)
last_index = get_last_index(OUTPUT_FILE_NAME)
print('start from ', last_index) #CHECK IF ITS LAST_INDEX + 1 NOW

In [23]:
with open(OUTPUT_FILE_NAME, 'a') as f:
    for _, row in X_test.iterrows():
        row_index = row['index']

        if row_index < last_index:
            continue
        predicted_value = predict(row['image_link'], row['entity_name'])
        f.write(f"{row_index},{predicted_value}\n")
        print(f"Appended index: {row_index} with predicted value: ")

Streaming output truncated to the last 5000 lines.
Appended index: 31862 with predicted value: 
[2024/09/15 06:59:51] ppocr DEBUG: dt_boxes num : 25, elapsed : 0.024346351623535156
[2024/09/15 06:59:51] ppocr DEBUG: cls num  : 25, elapsed : 0.0352480411529541
[2024/09/15 06:59:51] ppocr DEBUG: rec_res num  : 25, elapsed : 0.08635759353637695
['5.25 ounce']
Appended index: 31863 with predicted value: 
[2024/09/15 06:59:51] ppocr DEBUG: dt_boxes num : 6, elapsed : 0.059324026107788086
[2024/09/15 06:59:51] ppocr DEBUG: cls num  : 6, elapsed : 0.0108489990234375
[2024/09/15 06:59:51] ppocr DEBUG: rec_res num  : 6, elapsed : 0.021811485290527344
['18.0 centimetre', '7.09 inch', '110.0 centimetre', '43.3 inch', '8.5 centimetre', '3.35 inch', '40.0 centimetre', '15.75 inch']
Appended index: 31864 with predicted value: 
[2024/09/15 06:59:51] ppocr DEBUG: dt_boxes num : 5, elapsed : 0.053225040435791016
[2024/09/15 06:59:51] ppocr DEBUG: cls num  : 5, elapsed : 0.010465860366821289
[2024/09/15